In [1]:
import sys
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/Preprocess')
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/DAlgorithms')

import Preprocessdata
import MySVM as mysvc
import Superplot
from __future__ import print_function
from __future__ import division

import logging
from sklearn.datasets import make_classification
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC, SVR

from bayes_opt import BayesianOptimization
from pandas.core.frame import DataFrame
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.metrics.classification import accuracy_score, confusion_matrix, classification_report
from scipy.interpolate import spline
from sklearn import metrics
from sklearn.metrics import accuracy_score
from scipy import stats
from sklearn.gaussian_process import GaussianProcess
import timeit
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from IPython.core.pylabtools import figsize
from scipy.interpolate import spline
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
import re
from astropy.io.fits.header import Header
from matplotlib.pyplot import xlim
from sklearn import svm
%matplotlib inline

In [2]:

# Load data set and target values
start = timeit.default_timer()
save_path = '/home/peng/git/Machine_learning_for_reliability_analysis/Kernel/Results/Alloy600/'

data_path = '/home/peng/git/Machine_learning_for_reliability_analysis/Data/'

result_name = 'j1_rbf_origin_train_v1.csv'

####### Read the source data######################
df =pd.read_csv(data_path + 'alloy600_data.csv', header=0)


p= Preprocessdata.standardprocess()



seed = 'rbf'
rad_stat = 2
df = df.sample(frac = 1.0)

#train, trainlabel, test, testlabel = p.scaledivd(df, 0.8)
#print (np.shape(train))
print (df)

        T1    T2    T3       T4     T5     T6        T7     T8        Target
5    316.0  26.7  7.28   226.00 -0.810  334.0  1000.000   2.00  1.550000e-08
4    316.0  26.4  7.28   226.00 -0.810  334.0  1000.000   2.00  2.280000e-08
41   330.0  25.8  8.01   257.00 -0.909  389.0   300.000   2.20  3.630000e-08
114  325.0  32.5  7.79   232.00 -0.878  301.5   385.000   2.00  2.290000e-09
12   350.0  23.8  9.19   819.00 -1.096  389.0   108.000   7.00  3.780000e-08
136  326.9  21.0  7.46   379.00 -0.844  248.4  1764.000   3.37  1.780000e-09
123  290.0  31.9  7.26   232.00 -0.783  301.5   385.000   2.00  1.130000e-09
21   330.0  79.2  6.21    12.00 -0.693  389.0  1200.000   0.10  5.700000e-08
95   330.0  34.8  7.55   224.00 -0.856  394.0  1000.000   2.00  2.470000e-08
7    350.0  62.7  9.19   819.00 -1.096  389.0   108.000   7.00  3.540000e-07
116  325.0  32.1  7.79   232.00 -0.878  301.5   385.000   2.00  2.520000e-09
63   360.0  21.5  8.37   207.00 -1.000  468.0  1000.000   2.00  1.110000e-07

In [3]:
#### scale the training dataset first and then apply the rules to test set 
train, trainlabel, test, testlabel = p.scaledivd(df, 0.7)
print (np.shape(test))


(49, 8)


In [5]:
# use Bayesian to optimize the parameters
def rfccv(C_range, gamma_range, eps_range=0.0001):
    return cross_val_score(SVR(C=C_range, gamma = gamma_range, epsilon = eps_range),
                           train, trainlabel, 'r2', cv=10, n_jobs = -1).mean()

rfcBO = BayesianOptimization(rfccv, {'C_range': (1, 1000),                                         
                                         'gamma_range': (0.1, 2)})
rfcBO.maximize()

print('Final Results')
print('RF: %f' % rfcBO.res['max']['max_val'])

Initialization
---------------------------------------------------------
 Step |   Time |      Value |   C_range |   gamma_range | 
    1 | 00m00s |    0.30823 |  945.6355 |        0.8549 | 
    2 | 00m00s |    0.26139 |  136.6537 |        1.8359 | 
    3 | 00m00s |    0.61586 |  120.4673 |        0.4105 | 
    4 | 00m00s |    0.02868 |  825.3192 |        1.1722 | 
    5 | 00m00s |   -0.25693 |  907.0193 |        1.4840 | 
Bayesian Optimization
---------------------------------------------------------
 Step |   Time |      Value |   C_range |   gamma_range | 
    6 | 00m04s |    0.62126 |  385.3026 |        0.2639 | 
    7 | 00m01s |    0.80230 |  181.0274 |        0.1000 | 
    8 | 00m02s |    0.42450 |    1.0000 |        0.1000 | 
    9 | 00m01s |    0.81343 |  247.2944 |        0.1000 | 
   10 | 00m02s |    0.82482 |  876.6325 |        0.1000 | 
   11 | 00m02s |    0.82192 | 1000.0000 |        0.1000 | 
   12 | 00m03s |    0.59472 |  935.1896 |        0.2377 | 
   13 | 00m02s |    0

In [15]:
# get the best parameters for plotting r2 on training and test
if (1): ### True use the above best parameters, false use the recored best set
    C_best = np.int(rfcBO.res['max']['max_params']['C_range'])
    gamma_best= np.float(rfcBO.res['max']['max_params']['gamma_range'])
    print ('use the latest records')

else:
    C_best = 507.439
    gamma_best = 0.244 
    print ('use the historical records')


#bestmodel= RFC(n_estimators = tree_best, max_features = feature_best).fit(train,trainlabel)

print (C_best, gamma_best)

use the latest records
678 0.1


In [25]:
# use the new features to train

ff = mysvc.training_manCV()

df = ff.train_regress(train, trainlabel, 'rbf', Cmin=0, Cmax=20, numC=11, rmin=-10, rmax=10, numr=11, degree = 3,
                rad_stat = rad_stat, method= 'r2')

df.to_csv(save_path + result_name , header = True)


0.307108151027
1 cycle finished, 10 left
0.341770921227
2 cycle finished, 9 left
0.297269213604
3 cycle finished, 8 left
0.195440902915
4 cycle finished, 7 left
0.0873116373676
5 cycle finished, 6 left
-0.44022627188
6 cycle finished, 5 left
-1.56172420988
7 cycle finished, 4 left
-4.94276702804
8 cycle finished, 3 left
-11.808720492
9 cycle finished, 2 left
-40.1825276936
10 cycle finished, 1 left
-287.668468905
11 cycle finished, 0 left


In [27]:
#load the results from the saved file

df =pd.read_csv(save_path + result_name, header = 0)
df_new = df.drop('gamma_range',1)
df_new = df_new.drop('Unnamed: 0',1)

list_c = df_new.columns.values.tolist()
idx_C, idx_gamma = [],[]
for i in list_c:
    for j in xrange(0,11):
        if df[i][j]== (df_new.values.max()):
            
            idx_C.append(i)
            idx_gamma.append(j)
        else:
            pass
print (idx_C)
print (idx_gamma)    
# if only got unique maximum value, then check 

C_best = idx_C[0]
gamma_best = idx_gamma[0]

print (df[C_best][gamma_best], df_new.values.max())
C_best = np.float(C_best)
gamma_best = np.float(gamma_best)


['16384.0']
[2]
0.839807351407 0.839807351407


In [13]:
C_best = 507.439
gamma_best = 0.244 

In [28]:
#use the best parameters to plot cv auc on the training set
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cross_validation import StratifiedKFold


figsize(10,8)
#cv = StratifiedKFold(trainlabel, n_folds=10)
bestmodel= SVR(kernel=seed, C= C_best, gamma= gamma_best, epsilon=0.0001).fit(train, trainlabel)
#bestmodel_training= SVR(kernel=seed, C= C_best, gamma= gamma_best, epsilon=0.0001)
mean_mae = 0.0
mean_mse = 0.0
mean_r2 = 0.0

outputtest= bestmodel.predict(test)
mean_r2 = r2_score(testlabel, outputtest)
print (mean_r2)

for i, (training, testing) in enumerate(cv):
    
    #outputtest= bestmodel_training.fit(train[training], trainlabel[training]).predict(train[testing])
    #mean_mae = mean_mae +  mean_absolute_error(trainlabel[training], outputtest)
    #mean_mse = mean_mse +  mean_squared_error(trainlabel[training], outputtest)
    #mean_r2 = mean_r2 +  r2_score(trainlabel[training], outputtest)
    # Compute ROC curve and area the curve
    #int (np.shape(train[training]))
    #rint (np.shape(trainlabel[training]))
    pass

print ('MAE : ', mean_mae/10 )    
print ('MSE : ', mean_mse/10 ) 
print ('R2 : ', mean_r2/10 ) 

-1.92719358837
MAE :  0.0
MSE :  0.0
R2 :  -0.192719358837


In [24]:
#predict the test set

tt=mysvc.test()
result, socres = tt.test_regression(test, testlabel,bestmodel_training)

This is for test set**************************************
The MAE for the test set is 0.030549561304966468
The MSE for the test set is 0.0091371900722992724
The R2 for the test set is -0.49066745746836893


In [ ]:
#plot the confusion matrix on test set

from sklearn.metrics.classification import accuracy_score, confusion_matrix, classification_report
print (confusion_matrix(result, testlabel))
pp = Superplot.fancy()
pp.plot_confusion_matrix(confusion_matrix(result, testlabel))

In [ ]:
#plot the auc on test set
from sklearn.metrics import roc_curve, auc

y_score = bestmodel_training.predict_proba(test)
#print (y_score)

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

In [ ]:
#plot the auc on test set
fpr, tpr, thresholds = roc_curve(testlabel, y_score[:, 1])
roc_auc = auc(fpr, tpr)

figsize(10,8)

plt.plot(fpr, tpr, lw=1.5, label='(area= %0.2f)' % ( roc_auc))
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
#plt.xlim([-0.05, 1.05])
#plt.ylim([-0.05, 1.05])

plt.xlim([0, 1])
plt.ylim([0, 1])

plt.xlabel('False Positive Rate',fontsize = 24)
plt.ylabel('True Positive Rate',fontsize = 24)
plt.xticks(fontsize = 20) 

plt.yticks(fontsize = 20) 

#plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right",fontsize = 20)
sec = np.arange(0.0, 1.0, 0.05)
plt.fill_between(fpr, 0, tpr, color="#348ABD", alpha=0.3)
plt.show()

In [ ]:
df_score = pd.DataFrame({'y_origin': y_score[:,1]})
print (df_score)

In [ ]:
df_score.to_csv(save_path +'y_score_comp.csv', header = True)

In [ ]:
#plot the grid search


df = pd.read_csv(save_path+'j1_rbf_tree_train_auc_1.csv', header = 0)

scores=np.array(df)
scores=scores[:, 2:].T
#    print scores
#scores= scores[:,5:]
print (np.shape(scores))

#    print np.arange(100,2010,20)

figsize(8,6.5)
fig, ax = plt.subplots(1,1)
cax = ax.imshow(scores, interpolation='none', origin='highest',
                cmap=plt.cm.coolwarm, aspect=1)

plt.grid(b=False, which='x', color='white',linestyle='-')




plt.xticks(np.arange(-0.5,21.5,5), (-10,-5,0,5,10),fontsize = 20)
plt.yticks(np.arange(-0.5,23.5,5), (-10,-5,0,5,10),fontsize = 20)

#plt.yticks(np.arange(0,11,1), np.arange(1,12,1), fontsize = 20)

plt.ylabel('$log_2 C$',fontsize = 24)
plt.xlabel('$log_2 \gamma$', fontsize = 24)

ax.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
ax.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
ax.grid(b=True, which='major', color='w', linewidth=0.5)
ax.grid(b=True, which='minor', color='w', linewidth=0.5)


cb = fig.colorbar(cax)
cb.ax.tick_params(labelsize=14)


plt.show()